In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
Transition_matrix = load(fullname*".jld2","transition_matrix");
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [3]:
function reward(s, a,s1=10, rng=23)       # independent reward and action, without system-level cost decomposition
    r_c = 0.0
     if a==1 # don't replace component
        if Number_level>s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r_c = normal_operation ;
         return r_c
        else
            # r=system_penalty/NumberUnits;
            return r_c
        end
        end

    if a==2 # replace component
        if Number_level>s
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r_c = maintenance_penalty
            # r = maintenance_penalty +  setup_cost/NumberUnits;
         return r_c
        else
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r_c = failure_penalty
            # r= failure_penalty  +  setup_cost/NumberUnits;
        return r_c
        end
        end
    end


reward (generic function with 3 methods)

In [4]:
s=1:Number_level;

In [5]:
(Transition_matrix)

10×10 Matrix{Float64}:
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.0       0.332736  0.174857     0.0483409  0.0133439   0.00507506
 0.0       0.0       0.218499     0.0823996  0.0244349   0.00601449
 0.0       0.0       0.0          0.0926184  0.0592904   0.00740484
 0.0       0.0       0.0          0.174542   0.0815024   0.0129094
 0.0       0.0       0.0       …  0.254279   0.105602    0.0195799
 0.0       0.0       0.0          0.271986   0.158174    0.0867437
 0.0       0.0       0.0          0.589988   0.260696    0.149316
 0.0       0.0       0.0          0.0        0.519525    0.480475
 1.0       0.0       0.0          0.0        0.0         0.0

In [6]:
T=zeros(Number_level,Number_level,3);


In [7]:
((1:10)./10)

0.1:0.1:1.0

In [8]:
T[:,:,1].=Transition_matrix;

In [9]:
T[:,:,2].=Transition_matrix;


In [ ]:
for j in 1:Number_level
        T[j,:,3].=Transition_matrix[1,:];

        end


In [11]:
T[Number_level,:,1].=0.0
T[Number_level,Number_level,1]=1.0
T[Number_level,:,2].=Transition_matrix[1,:]


10-element view(::Array{Float64, 3}, 10, :, 2) with eltype Float64:
 0.2695279572108137
 0.16860741623354286
 0.1306613209439592
 0.09857168678543618
 0.09171796107138079
 0.09134806335671229
 0.09048875946531719
 0.04832973229444604
 0.00607725632113932
 0.004669846317252394

In [13]:
T

10×10×3 Array{Float64, 3}:
[:, :, 1] =
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.0       0.332736  0.174857     0.0483409  0.0133439   0.00507506
 0.0       0.0       0.218499     0.0823996  0.0244349   0.00601449
 0.0       0.0       0.0          0.0926184  0.0592904   0.00740484
 0.0       0.0       0.0          0.174542   0.0815024   0.0129094
 0.0       0.0       0.0       …  0.254279   0.105602    0.0195799
 0.0       0.0       0.0          0.271986   0.158174    0.0867437
 0.0       0.0       0.0          0.589988   0.260696    0.149316
 0.0       0.0       0.0          0.0        0.519525    0.480475
 0.0       0.0       0.0          0.0        0.0         1.0

[:, :, 2] =
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.0       0.332736  0.174857     0.0483409  0.0133439   0.00507506
 0.0       0.0       0.218499     0.0823996  0.0244349   0.00601449
 0.0       0.0       0.0          0.0926184  0.0592904   0.00740484
 0.0      

In [14]:
T[:, :, 3]

10×10 Matrix{Float64}:
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661  …  0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985
 0.269528  0.168607  0.130661     0.0483297  0.00607726  0.00466985

In [ ]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       # calculate true component-level reward r_i(s_i,a_i) nd get evolving states
    if a==1  # don't replace component
            crd = Categorical(T[s,:,1]);
            s = rand(crd);
            if Number_level>s
                crd = Categorical(Transition_matrix[s,:]);
                s = rand(crd);
                r = normal_operation ;
                return (sp=s, r=r)
            else
                r=0;

             return (sp=s, r=r)
 
        end
    end
    if a==2 # replace component
        if s==Number_level
             s=1
            crd = Categorical(T[s,:,2]);
            s = rand(crd);
            r=failure_penalty  + normal_operation  ;
        return (sp=s, r=r)
        else
            crd = Categorical(T[s,:,2]);
            s = rand(crd);
            r = normal_operation;
             return (sp=s, r=r)
        end
    end

     
end


generative (generic function with 2 methods)

In [ ]:
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,2))
#     print(Q)
    V= zeros(Float64, (Number_level)).+1000
    diff=0
    i=0

    
   
    while i<=10000

    for s in 1:Number_level
       Threads.@threads for a in 1:2

                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:Number_level ))
            
        end

                V[s]=findmax(Q[s,1:2])[1]
                 
        end

        i+=1
        end
    return Q
    end

ValueIteration (generic function with 3 methods)

In [17]:
if(1==1)&&(1>=0)
    print("yes")
end

yes

In [ ]:
function decison(Q, s,n=0.2)
   a_final=[]

   if count(i->(i==Number_level), s) >=limit

        a_1=repeat(1:1,NumberUnits);
        for i in 1:NumberUnits
            a_1[i]=argmax(a->Q[s[i],a], [1,2])
            
            if s[i]==Number_level 
                a_1[i]=2
               
            end
        end
        return a_1
    end
    
    
    
    
    
    q0=0.0
    a_0=repeat(1:1,NumberUnits);
    for i in 1:NumberUnits
            q0+=Q[s[i],1]
    end
    a_1=repeat(1:1,NumberUnits);
    b=0
    for i in 1:NumberUnits
            a_1[i]=argmax(a->Q[s[i],a], [1,2])
            if s[i]==Number_level
                a_1[i]=2
            end
    end
    q1=0.0
    for i in 1:NumberUnits
            q1+=Q[s[i],a_1[i]]
    end

    if (2 in a_1)
        q1 += setup_cost # add setup cost since not included at component level
    end
    
    if !(2 in a_1) 
        a_1=a_0
        q1=q0
    end
    
    
     if q0>q1
            return a_0
        else
#             print("HI")
            return a_1
            
        end
end

decison (generic function with 2 methods)

In [19]:

using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])
cost=[ 
    [0	-2000	-100	-1200 -2000]
] 



for units in [20, 25, 30, 40, 50, 60]
    for K in [6]
        
    global NumberUnits=convert(Int64,units)
        global limit=convert(Int64,K)
#      for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
     for lambda in 1:1:1
#      for lambda1 in [0.0001,0.00001]
        
        global lamb=lambda
#         global lamb1=lambda1
            
            for c in cost
        tick()
   println(units)  
             print("K : ")
                    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println(c)   

    global NumberUnits=units
     Q=ValueIteration(s,T,.95,3000)
#                 println(Q)
            println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
# Iteration
 for k in 1:simsteps
    a =decison(Q,s)
            r2=0.0
#     print(s)
#                         println(a)
global failed_components = 0
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
        if s[i] == Number_level
            failed_components += 1
        end
        s[i],r[i]=generative(s[i],a[i],234);
        r2+=r[i]
    end 
    if  (2 in a) 
        # if any replacement
            r2+=setup_cost
                    end
    if failed_components >= limit
        r2+=system_penalty
    end
#     println(r2)
#     println(s)
    s=s;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end
end
end

20
K : 6


[ Info:  started timer at: 2025-07-24T20:11:58.162


[0 -2000 -100 -1200 -2000]
false


[ Info:  started timer at: 2025-07-24T20:12:13.856


20
k : 6
1
-128261.78695684896
4911.979922592777
-------------------------
25
K : 6
[0 -2000 -100 -1200 -2000]


...

false


[ Info:  started timer at: 2025-07-24T20:19:50.718


60
k : 6
1
-371854.34707665537
5372.182925850589
-------------------------


[ Info:          137.7884555s: 2 minutes, 17 seconds, 788 milliseconds


LoadError: ParseError:
[90m# Error @ [0;0m]8;;file://C:/Users/hhu98/OneDrive - UW-Madison/K-out-of-N 4th rebuttal/In[19]#98:1\[90mIn[19]:98:1[0;0m]8;;\
end
[48;2;120;70;70mend[0;0m
[90m└─┘ ── [0;0m[91minvalid identifier[0;0m

In [20]:
df

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,60.0,6.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-3.71854e5,5372.18
2,50.0,6.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-3.20361e5,5480.41
3,40.0,6.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-2.64812e5,6461.87
4,30.0,6.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-1.9785e5,6861.89
5,25.0,6.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-1.62529e5,5818.4
6,20.0,6.0,0.0,-2000.0,-100.0,-1200.0,-2000.0,1.0,-1.28262e5,4911.98


In [21]:
using CSV
CSV.write("Independent solutions with system-level enumeration.csv", df)

"Independent solutions with system-level enumeration.csv"